In [ ]:
%pip install Faker

### Import libraries

In [ ]:
import dg_functions
import json
import boto3
import initialize as cp
import uuid

### Generate Hotel Loyalty, Preferences, Hotel Bookings and Stay Revenue Objects

In [ ]:
dataObj = dg_functions.generateHotelData()
print(f"LoyaltyID: {dataObj['Loyalty']['LoyaltyId']}")
print(f"No of Bookings: {len(dataObj['Booking'])}")
print(f"No of Stay Revenue: {len(dataObj['StayRevenue'])}")

### Send the Loyalty Object to Customer Profiles

In [ ]:
cp.client.put_profile_object(
    DomainName = cp.domainName,
    ObjectTypeName = 'HotelLoyalty',
    Object = json.dumps(dataObj['Loyalty'])
)

### Send the preferences data to Customer Profiles

In [ ]:
cp.client.put_profile_object(
    DomainName = cp.domainName,
    ObjectTypeName = 'HotelPreferences',
    Object = json.dumps(dataObj['Preferences'])
)

### Send the booking records to Kinesis

In [ ]:
kinesis_client = boto3.client('kinesis')
stream_name = 'hotel-booking'
partition_key = str(uuid.uuid4())

In [ ]:
# Convert data to JSON and then encode to bytes
bookings = dataObj['Booking']
for booking in bookings:
    data_as_bytes = json.dumps(booking).encode('utf-8')
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=data_as_bytes,
        PartitionKey=partition_key
    )

### Send the Hotel Stay Revenue to Customer Profiles

In [ ]:
for stayReservation in dataObj['StayRevenue']:
    cp.client.put_profile_object(
        DomainName = cp.domainName,
        ObjectTypeName = 'HotelStayRevenue',
        Object = json.dumps(stayReservation)
    )